In [50]:
import os
import PIL
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision.utils import make_grid, save_image
from torchvision import transforms, datasets
import cv2
from utils import visualize_cam, Normalize
from gradcam import GradCAM, GradCAMpp
from efficientnet_pytorch import EfficientNet

In [51]:
def show(image):
    cv2.imshow("title",image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [52]:
def make_grid2(net_name, output_dir,data_name):
    normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    if net_name == 'vgg':
        net_model_dict = dict(type='vgg', arch=vgg, layer_name='features_29', input_size=(224, 224))
    elif net_name == 'alexnet':
        net_model_dict = dict(type='alexnet', arch=alexnet, layer_name='features_11', input_size=(224, 224))
    elif net_name == 'resnet':
        net_model_dict = dict(type='resnet', arch=resnet, layer_name='layer4', input_size=(224, 224))
    elif net_name == 'densenet':
        net_model_dict = dict(type='densenet', arch=densenet, layer_name='features_norm5', input_size=(224, 224))
    elif net_name == 'squeezenet':
        net_model_dict = dict(type='squeezenet', arch=squeezenet, layer_name='features_12_expand3x3_activation', input_size=(224, 224))
    elif net_name == 'efficientnet':
        net_model_dict = dict(type='efficientnet', arch=efficientnet, layer_name='_blocks 54 _bn2', input_size=(224, 224))

    for i in range(16):
        img_name = files[i]
        img_path = os.path.join(img_dir, img_name)

        pil_img = PIL.Image.open(img_path)

        if len(np.asarray(pil_img).shape) == 3:
            torch_img = torch.from_numpy(np.asarray(pil_img)).permute(2, 0, 1).unsqueeze(0).float().div(255)
        else :
            np_img = np.asarray(pil_img)
            np_img = cv2.cvtColor(np_img,cv2.COLOR_GRAY2RGB)
            torch_img = torch.from_numpy(np_img).permute(2, 0, 1).unsqueeze(0).float().div(255)
        torch_img = F.upsample(torch_img, size=(224, 224), mode='bilinear', align_corners=False)
        normed_torch_img = normalizer(torch_img)

        gradcam_pp = GradCAMpp(net_model_dict, True)
        mask_pp, _ = gradcam_pp(normed_torch_img)
        heatmap_pp, result_pp = visualize_cam(mask_pp, torch_img)

        file_name = '{}.jpeg'.format(i)
        output_path = os.path.join(output_dir, file_name)
        save_image(result_pp,output_path)

    images = []
    trans1 = transforms.ToTensor()

    for i in range(16):
        file_name = '{}.jpeg'.format(i)
        output_path = os.path.join(output_dir, file_name)
        images.append(torch.stack([trans1(PIL.Image.open(output_path))], 0))
        
    images = make_grid(torch.cat(images, 0), nrow=4)
    
    if net_name == 'vgg':
        grid_output_path = os.path.join(output_dir, "grid_v_{}.jpeg".format(data_name))
        save_image(images, grid_output_path)
    elif net_name == 'alexnet':
        grid_output_path = os.path.join(output_dir, "grid_a_{}.jpeg".format(data_name))
        save_image(images, grid_output_path)
    elif net_name == 'resnet':
        grid_output_path = os.path.join(output_dir, "grid_r_{}.jpeg".format(data_name))
        save_image(images, grid_output_path)
    elif net_name == 'densenet':
        grid_output_path = os.path.join(output_dir, "grid_d_{}.jpeg".format(data_name))
        save_image(images, grid_output_path)
    elif net_name == 'squeezenet':
        grid_output_path = os.path.join(output_dir, "grid_s_{}.jpeg".format(data_name))
        save_image(images, grid_output_path)
    elif net_name == 'efficientnet':
        grid_output_path = os.path.join(output_dir, "grid_e_{}.jpeg".format(data_name))
        save_image(images, grid_output_path)

### data directory

In [53]:
efficientnet = EfficientNet.from_pretrained('efficientnet-b7',num_classes=2)
efficientnet.load_state_dict(torch.load("pth/0819/efficientnetch.pth",map_location='cpu'))

alexnet = models.alexnet()
num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs, 2)
alexnet.load_state_dict(torch.load("pth/0819/alexnetch.pth",map_location='cpu')) # use my model parameter and use cpu
alexnet.eval()

vgg = models.vgg16()
vgg.classifier[6] = nn.Linear(4096,2)
vgg.load_state_dict(torch.load("pth/0819/vgg16ch.pth",map_location='cpu')) # use my model parameter and use cpu
vgg.eval()

resnet = models.resnet18()
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 2)
resnet.load_state_dict(torch.load("pth/0819/resnet18ch.pth",map_location='cpu'))

densenet = models.densenet161()
densenet.classifier = nn.Linear(2208, 2)
densenet.load_state_dict(torch.load("pth/0819/densenet161ch.pth",map_location='cpu')) # use my model parameter and use cpu
densenet.eval()

squeezenet = models.squeezenet1_1()
squeezenet.classifier[1]  = nn.Conv2d(512, 2, kernel_size=(1,1), stride=(1,1))
squeezenet.num_classes = 2
squeezenet.load_state_dict(torch.load("pth/0819/squeezenet1_1ch.pth",map_location='cpu'))
squeezenet.eval()

Loaded pretrained weights for efficientnet-b7


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [ ]:
img_dir = 'dataset/chest_xray/val/PNEUMONIA/'
files = os.listdir(img_dir)

make_grid2('vgg','outputs/chest/pneumonia/','ch')
make_grid2('alexnet','outputs/chest/pneumonia/','ch')
make_grid2('resnet','outputs/chest/pneumonia/','ch')
make_grid2('densenet','outputs/chest/pneumonia/','ch')
make_grid2('squeezenet','outputs/chest/pneumonia/','ch')
make_grid2('efficientnet','outputs/chest/pneumonia/','ch')

In [ ]:
classes = {int(0):'NORMAL', int(1):'PNEUMONIA'}
logit = efficientnet(normed_torch_img)
h_x = F.softmax(logit, dim=1).data.squeeze()
probs, idx = h_x.sort(0, True)
probs = probs.numpy()
idx = idx.numpy()
for i in range(0, 2):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

## dog&cat

In [47]:
efficientnet = EfficientNet.from_pretrained('efficientnet-b7',num_classes=2)
efficientnet.load_state_dict(torch.load("pth/0819/efficientnetcd.pth",map_location='cpu'))

alexnet = models.alexnet()
num_ftrs = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_ftrs, 2)
alexnet.load_state_dict(torch.load("pth/0819/alexnetcd.pth",map_location='cpu')) # use my model parameter and use cpu
alexnet.eval()

vgg = models.vgg16()
vgg.classifier[6] = nn.Linear(4096,2)
vgg.load_state_dict(torch.load("pth/0819/vgg16cd.pth",map_location='cpu')) # use my model parameter and use cpu
vgg.eval()

resnet = models.resnet18()
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 2)
resnet.load_state_dict(torch.load("pth/0819/resnet18cd.pth",map_location='cpu'))

densenet = models.densenet161()
densenet.classifier = nn.Linear(2208, 2)
densenet.load_state_dict(torch.load("pth/0819/densenet161cd.pth",map_location='cpu')) # use my model parameter and use cpu
densenet.eval()

squeezenet = models.squeezenet1_1()
squeezenet.classifier[1]  = nn.Conv2d(512, 2, kernel_size=(1,1), stride=(1,1))
squeezenet.num_classes = 2
squeezenet.load_state_dict(torch.load("pth/0819/squeezenet1_1cd.pth",map_location='cpu'))
squeezenet.eval()

Loaded pretrained weights for efficientnet-b7


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): MaxPool2d

In [ ]:
img_dir = 'dataset/cat-and-dog/val/cats/' #change!
files = os.listdir(img_dir)

make_grid2('vgg','outputs/cat_and_dog/cat/','cd')
make_grid2('alexnet','outputs/cat_and_dog/cat/','cd')
make_grid2('resnet','outputs/cat_and_dog/cat/','cd')
make_grid2('densenet','outputs/cat_and_dog/cat/','cd')
make_grid2('squeezenet','outputs/cat_and_dog/cat/','cd')
make_grid2('efficientnet','outputs/cat_and_dog/cat/','cd')

### for accuracy

In [54]:
def get_acc(model,net_name):
    corrects = 0
    for inputs, labels in dataloaders['val']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        # statistics
        corrects += torch.sum(preds == labels.data)

    acc = corrects.double() / dataset_sizes['val']

    print('{}: Acc: {:.4f}'.format(net_name, acc))

In [55]:
data_dir = 'dataset/chest_xray/'

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

get_acc(alexnet,'alexnet')
get_acc(squeezenet,'squeezenet')
get_acc(vgg,'vgg')
get_acc(resnet,'resnet')

alexnet: Acc: 0.9250
squeezenet: Acc: 0.9297
vgg: Acc: 0.9391
resnet: Acc: 0.9172


In [56]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [58]:
print("alexnet : {}".format(count_parameters(alexnet)))
print("squeezenet : {}".format(count_parameters(squeezenet)))
print("vgg : {}".format(count_parameters(vgg)))
print("resnet : {}".format(count_parameters(resnet)))
print("densenet : {}".format(count_parameters(densenet)))
print("efficientnet : {}".format(count_parameters(efficientnet)))

alexnet : 57012034
squeezenet : 723522
vgg : 134268738
resnet : 11177538
densenet : 26476418
efficientnet : 63792082
